In [ ]:
import folium
import numpy as np
import pandas as pd
from folium.plugins import HeatMap
from folium import plugins
from folium import GeoJson
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.colors as mcolors
import plotly.express as px
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
import os

#######################################################################
data_small_df = pd.read_csv("data/fullWithLocations_FINAL_small.csv")
data_large_df = pd.read_csv("data/fullWithLocations_FINAL_large.csv")
#######################################################################
group_small = [
    "AIRMASS", "ALLSKY_SFC_SW_DWN", "ALLSKY_SFC_UVA", "ALLSKY_SFC_UVB",
    "ALLSKY_SFC_UV_INDEX", "CLOUD_AMT", "CLOUD_AMT_DAY", "CLOUD_AMT_NIGHT",
    "CLRSKY_DAYS", "MIDDAY_INSOL", "PSH", "PW"
]

group_large = [
    " EVLAND", "GWETPROF", "GWETROOT", "GWETTOP", "PRECSNO", "PRECTOTCORR",
    "QV2M", "RH2M", "RHOA", "T10M", "T10M_MAX", "T10M_MIN", "T2M",
    "T2M_MAX", "T2M_MIN", "TO3", "TS", "TSOIL1", "TSOIL2", "TSOIL3",
    "TSOIL4", "TSOIL5", "TSOIL6", "TS_MAX", "TS_MIN", "WD2M", "WD50M",
    "WS2M", "WS2M_MAX", "WS2M_MIN", "WS50M", "WS50M_MAX", "WS50M_MIN", "Z0M"
]


In [63]:
loc_Small=data_small_df.drop_duplicates()
loc_Large=data_large_df.drop_duplicates()

print(loc_Small.head())
print(loc_Large.head())

   Unnamed: 0   LAT   LON  YEAR  DOY  AIRMASS  ALLSKY_SFC_SW_DWN  \
0           1  22.5  25.5  2019    1     3.93              14.94   
1           2  22.5  26.5  2019    1     4.02              13.70   
2           3  22.5  27.5  2019    1     4.14              12.29   
3           4  22.5  28.5  2019    1     4.29              11.38   
4           5  22.5  29.5  2019    1     4.46              11.21   

   ALLSKY_SFC_UVA  ALLSKY_SFC_UVB  ALLSKY_SFC_UV_INDEX  ...  CLOUD_AMT_NIGHT  \
0            0.81            0.02                 1.12  ...            19.19   
1            0.75            0.02                 1.04  ...            16.39   
2            0.67            0.01                 0.91  ...            29.16   
3            0.62            0.01                 0.85  ...            52.74   
4            0.59            0.01                 0.80  ...            74.05   

   CLRSKY_DAYS  MIDDAY_INSOL   PSH    PW               GOV  \
0            0         64.97  0.36  0.69  Al Wad

In [64]:

loc_Large = loc_Large[loc_Large["LAT"].isin(loc_Small["LAT"])]
mask = loc_Large["LON"].apply(lambda x: np.any(np.abs(loc_Small["LON"] - x) <= 0.25))
loc_Large = loc_Large[mask]

In [65]:
coppy=loc_Large.copy()

In [45]:
display(loc_Large[["LAT","LON"]].drop_duplicates().head(10))
display(loc_Small[["LAT","LON"]].drop_duplicates().head(10))
display(len(loc_Large))

,LAT,LON
0,22.5,25.625
1,22.5,26.250
3,22.5,27.500
5,22.5,28.750
6,22.5,29.375
8,22.5,30.625
9,22.5,31.250
11,22.5,32.500
13,22.5,33.750
14,22.5,34.375


,LAT,LON
0,22.5,25.5
1,22.5,26.5
2,22.5,27.5
3,22.5,28.5
4,22.5,29.5
5,22.5,30.5
6,22.5,31.5
7,22.5,32.5
8,22.5,33.5
9,22.5,34.5


166166

In [ ]:
loc_Large['LON'] = loc_Large['LON'].astype(float)

lon_mapping = {
    25.625: 25.5,
    26.25: 26.5,
    27.5: 27.5,
    28.75: 28.5,
    29.375: 29.5,
    30.625: 30.5,
    31.25: 31.5,
    32.5: 32.5,
    33.75: 33.5,
    34.375: 34.5,
    35.625: 35.5
}
loc_Large['LON'].replace(lon_mapping, inplace=True)
loc_Large.dropna()

C:\Users\ahmed\AppData\Local\Temp\ipykernel_5972\642326583.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  loc_Large['LON'].replace(lon_mapping, inplace=True)


,Unnamed: 0,LAT,LON,YEAR,DOY,EVLAND,GWETPROF,GWETROOT,GWETTOP,PRECSNO,...,WS2M_MIN,WS50M,WS50M_MAX,WS50M_MIN,Z0M,GOV,City,Date,Week,Month
0,17,22.5,25.5,2019,1,0.00,0.37,0.36,0.09,0.0,...,1.45,6.02,7.70,4.00,0.01,Al Wadi al Jadid,Shurtah al-Dakhlah,2019-01-01,1,1
1,18,22.5,26.5,2019,1,0.00,0.27,0.27,0.04,0.0,...,1.61,6.40,8.18,4.07,0.01,Al Wadi al Jadid,Shurtah al-Dakhlah,2019-01-01,1,1
3,20,22.5,27.5,2019,1,0.00,0.15,0.15,0.04,0.0,...,1.63,5.84,7.65,3.69,0.01,Al Wadi al Jadid,Shurtah al-Dakhlah,2019-01-01,1,1
5,22,22.5,28.5,2019,1,0.00,0.20,0.20,0.04,0.0,...,1.33,5.31,7.91,2.82,0.01,Al Wadi al Jadid,Shurtah al-Dakhlah,2019-01-01,1,1
6,23,22.5,29.5,2019,1,0.00,0.20,0.20,0.04,0.0,...,1.26,5.08,7.49,2.77,0.01,Al Wadi al Jadid,Shurtah al-Dakhlah,2019-01-01,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503958,122591,30.5,34.5,2023,365,0.19,0.47,0.47,0.34,0.0,...,0.80,5.02,7.82,1.66,0.01,Shamal Sina',Shurtah al-Qasimah,2023-12-31,52,12
503972,122609,31.5,25.5,2023,365,0.14,0.35,0.35,0.28,0.0,...,0.74,3.23,4.69,1.07,0.01,Matrouh,Sidi Barrani,2023-12-31,52,12
503973,122610,31.5,26.5,2023,365,0.09,0.34,0.33,0.25,0.0,...,0.54,3.40,4.90,0.74,0.00,Matrouh,Sidi Barrani,2023-12-31,52,12
503974,122617,31.5,30.5,2023,365,0.92,0.51,0.51,0.66,0.0,...,1.33,4.27,5.71,1.86,0.01,Kafr ash Shaykh,Sidi Salim,2023-12-31,52,12


In [69]:
egypt_map = folium.Map(location=[26.5, 30], zoom_start=6)
for _, row in loc_Large[["LAT","LON"]].drop_duplicates(). iterrows():
    folium.Circle(
        location=[row['LAT'], row['LON']], 
        radius=5000,  
        color='blue',
        fill=False,
        fill_opacity=0.1,
        weight=1
    ).add_to(egypt_map)
for _, row in loc_Small[["LAT","LON"]].drop_duplicates(). iterrows():
    folium.Circle(
        location=[row['LAT'], row['LON']],  
        radius=6000, 
        color='red',
        fill=False,
        fill_opacity=0.1,
        weight=1.5
    ).add_to(egypt_map)
for _, row in loc_Large[["LAT","LON"]].drop_duplicates(). iterrows():
    folium.Rectangle(
        bounds=[
            [row['LAT'] - 0.25, row['LON'] - 0.3125], 
            [row['LAT'] + 0.25, row['LON'] + 0.3125]   
        ],
        color='black',
        fill=True,
        fill_opacity=0.4,
        weight=1).add_to(egypt_map)   
egypt_map

In [70]:
print("df_small ",loc_Small.columns)
print("df_large ",loc_Large.columns)

df_small  Index(['Unnamed: 0', 'LAT', 'LON', 'YEAR', 'DOY', 'AIRMASS',
       'ALLSKY_SFC_SW_DWN', 'ALLSKY_SFC_UVA', 'ALLSKY_SFC_UVB',
       'ALLSKY_SFC_UV_INDEX', 'CLOUD_AMT', 'CLOUD_AMT_DAY', 'CLOUD_AMT_NIGHT',
       'CLRSKY_DAYS', 'MIDDAY_INSOL', 'PSH', 'PW', 'GOV', 'City', 'Date',
       'Week', 'Month'],
      dtype='object')
df_large  Index(['Unnamed: 0', 'LAT', 'LON', 'YEAR', 'DOY', 'EVLAND', 'GWETPROF',
       'GWETROOT', 'GWETTOP', 'PRECSNO', 'PRECTOTCORR', 'QV2M', 'RH2M', 'RHOA',
       'T10M', 'T10M_MAX', 'T10M_MIN', 'T2M', 'T2M_MAX', 'T2M_MIN', 'TO3',
       'TS', 'TSOIL1', 'TSOIL2', 'TSOIL3', 'TSOIL4', 'TSOIL5', 'TSOIL6',
       'TS_MAX', 'TS_MIN', 'WD2M', 'WD50M', 'WS2M', 'WS2M_MAX', 'WS2M_MIN',
       'WS50M', 'WS50M_MAX', 'WS50M_MIN', 'Z0M', 'GOV', 'City', 'Date', 'Week',
       'Month'],
      dtype='object')


In [ ]:
df_small = loc_Small.drop(columns=['Unnamed: 0'], errors='ignore')
df_large = loc_Large.drop(columns=['Unnamed: 0'], errors='ignore')

merged_df = pd.merge(df_small, df_large, on=['LAT', 'LON', 'DOY', 'YEAR'], how='inner')

merged_df = merged_df.drop_duplicates(subset=['LAT', 'LON', 'DOY', 'YEAR'])


In [72]:
print(len(merged_df))

164340


In [74]:
merged_df.to_csv("data/Applications.csv",index=False)

In [116]:
df=pd.read_csv("data/Applications.csv",index_col=False)

In [117]:
df.head()

,LAT,LON,YEAR,DOY,AIRMASS,ALLSKY_SFC_SW_DWN,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,CLOUD_AMT,...,TS_MIN,WD2M,WD50M,WS2M,WS2M_MAX,WS2M_MIN,WS50M,WS50M_MAX,WS50M_MIN,Z0M
0,22.5,25.5,2019,1,3.93,14.94,0.81,0.02,1.12,39.29,...,1.39,323.8,325.9,2.82,5.11,1.45,6.02,7.70,4.00,0.01
1,22.5,26.5,2019,1,4.02,13.70,0.75,0.02,1.04,42.90,...,2.33,315.7,317.8,3.01,5.05,1.61,6.40,8.18,4.07,0.01
2,22.5,27.5,2019,1,4.14,12.29,0.67,0.01,0.91,55.86,...,4.83,323.3,323.8,2.52,3.87,1.63,5.84,7.65,3.69,0.01
3,22.5,28.5,2019,1,4.29,11.38,0.62,0.01,0.85,69.87,...,4.58,317.5,322.6,2.28,3.44,1.33,5.31,7.91,2.82,0.01
4,22.5,29.5,2019,1,4.46,11.21,0.59,0.01,0.80,81.09,...,5.41,312.0,317.8,2.12,3.40,1.26,5.08,7.49,2.77,0.01


In [ ]:
Mango_df_Locations = df[
    # Wind parameters (use only if available)
    (df["WS2M"].between(1.0, 2.5))  &
    (df["WS2M_MAX"].between(2.5, 4.0)) & 
    (df["WS2M_MIN"].between(0.5, 1.0)) &
    
    # # Temperature parameters
    (df["TS"].between(25, 35)) &
    (df["TS_MAX"].between(30, 40)) & 
    (df["TS_MIN"].between(20, 25)) &
    (df["T2M"].between(25, 35)) &
    (df["T2M_MAX"].between(30, 40)) & 
    (df["T2M_MIN"].between(20, 25)) & 
    (df["T10M"].between(22, 28)) & 
    (df["T10M_MAX"].between(28, 35)) & 
    (df["T10M_MIN"].between(18, 22)) &
    
    # Humidity/moisture parameters
    (df["RH2M"].between(60, 70)) & 
    (df["QV2M"].between(0.014, 0.020)) &
    
    # Soil parameters
    (df["TSOIL1"].between(25, 28)) &
    (df["TSOIL2"].between(28, 32)) &
    (df["TSOIL3"].between(25, 28)) &
    (df["TSOIL4"].between(23, 25)) &
    (df["TSOIL5"].between(21, 23)) &
    (df["TSOIL6"].between(19, 21)) &
    
    # Water availability
    (df["PRECTOTCORR"].between(700, 900)) & 
    (df["GWETPROF"].between(0.7, 0.9)) & 
    (df["GWETROOT"].between(0.6, 0.8)) & 
    (df["GWETTOP"].between(0.8, 1.0)) &
    
   # Solar/radiation parameters
    (df["ALLSKY_SFC_SW_DWN"].between(5.5, 7.0)) & 
    (df["ALLSKY_SFC_UV_INDEX"].between(6, 8)) & 
    (df["ALLSKY_SFC_UVA"].between(30, 40)) & 
    (df["ALLSKY_SFC_UVB"].between(2.5, 3.5)) 
]

# Get unique coordinates
suitable_locations_For_Mango = Mango_df_Locations[["LON", "LAT"]].drop_duplicates()

print(f"Found {len(suitable_locations_For_Mango)} suitable mango-growing locations:")
print(suitable_locations_For_Mango)


Found 0 suitable mango-growing locations:
Empty DataFrame
Columns: [LON, LAT]
Index: []
